In [1]:
import os

In [26]:
%pwd

'E:\\Plant_disease_data\\Corn__healthy\\CompressedImages'

In [29]:
os.chdir(r"E:/Plant_disease_data/Corn__healthy/")

In [30]:
%pwd

'E:\\Plant_disease_data\\Corn__healthy'

In [28]:
%pip install Pillow

In [24]:
from PIL import Image

In [31]:
IMAGES = os.listdir(os.getcwd())
os.makedirs("./CompressedImages/", exist_ok= True)
new_width = 400  # Adjust to your desired width
new_height = 400  # Adjust to your desired height

for image in IMAGES:
    # Open the image
    original_image = Image.open(image)

    os.chdir(r"./CompressedImages/")
    # Resize the image to a new width and height
    resized_image = original_image.resize((new_width, new_height), Image.LANCZOS)
    # Compress the image and save it with a new name
    resized_image.save(f'compressed_{image}', optimize=True, quality=80)

    os.chdir(r"../")
    
os.chdir(r"./CompressedImages/")
for image in os.listdir(os.getcwd()):
    print(image)

compressed_20200610_063438.jpg
compressed_20200610_063444.jpg
compressed_20200610_063447.jpg
compressed_20200610_063449.jpg
compressed_20200610_063456.jpg
compressed_20200610_063507.jpg
compressed_20200610_063514.jpg
compressed_20200610_063522.jpg
compressed_20200610_063528.jpg
compressed_20200610_063534.jpg
compressed_20200610_063539.jpg
compressed_20200610_063541.jpg
compressed_20200610_063656.jpg
compressed_20200610_063709.jpg
compressed_20200610_063711.jpg
compressed_20200610_063713.jpg
compressed_20200610_063717.jpg
compressed_20200610_063721.jpg
compressed_20200610_063725.jpg
compressed_20200610_063729.jpg
compressed_20200610_063733.jpg
compressed_20200610_063737.jpg
compressed_20200610_063738.jpg
compressed_20200610_063752.jpg
compressed_20200610_063757.jpg
compressed_20200610_063802.jpg
compressed_20200610_063809.jpg
compressed_20200610_063813.jpg
compressed_20200610_063824.jpg
compressed_20200610_063828.jpg
compressed_20200610_063832.jpg
compressed_20200610_063834.jpg
compress

In [32]:
os.chdir(r"../")

In [33]:
%pwd

'E:\\Plant_disease_data\\Corn__healthy'